<a href="https://colab.research.google.com/github/LuckyHusbando/DI-Bootcamp/blob/main/Week9D2XP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Comprehensive Course on Advanced Data Visualization Techniques and SQLAlchemy
#Exercise 1

### download and extract chinook sample DB
import urllib.request
import zipfile
from functools import partial
import os
import sqlalchemy

chinook_url = 'http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip'
if not os.path.exists('chinook.zip'):
    print('downloading chinook.zip ', end='')
    with urllib.request.urlopen(chinook_url) as response:
        with open('chinook.zip', 'wb') as f:
            for data in iter(partial(response.read, 4*1024), b''):
                print('.', end='', flush=True)
                f.write(data)

zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')

### open the database using sqlalchemy module interface
engine = sqlalchemy.create_engine('sqlite:///chinook.db')
cur = engine.connect()


### useful: functions for displaying results from sql queries using pandas
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)

### useful: extract classes from the chinook database
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

## we need to do this once
from sqlalchemy.ext.automap import automap_base

# produce a set of mappings from this MetaData.
Base = automap_base(metadata=metadata)

# calling prepare() just sets up mapped classes and relationships.
Base.prepare()

# also prepare an orm session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

downloading chinook.zip ...........................................................................

In [3]:
inspector = sqlalchemy.inspect(engine)
table_names = inspector.get_table_names()
print("Tables in the database:")
for table_name in table_names:
    print(table_name)

Tables in the database:
albums
artists
customers
employees
genres
invoice_items
invoices
media_types
playlist_track
playlists
tracks


In [4]:
query = "SELECT * FROM tracks LIMIT 3;"
display_results(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99



SELECT * FROM tracks LIMIT 3;



In [5]:
query = """
SELECT
    t.Name AS TrackName,
    a.Title AS AlbumTitle
FROM
    tracks AS t
JOIN
    albums AS a ON t.AlbumId = a.AlbumId
LIMIT 20;
"""
display_results(query)

,TrackName,AlbumTitle
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You
1,Put The Finger On You,For Those About To Rock We Salute You
2,Let's Get It Up,For Those About To Rock We Salute You
3,Inject The Venom,For Those About To Rock We Salute You
4,Snowballed,For Those About To Rock We Salute You
5,Evil Walks,For Those About To Rock We Salute You
6,C.O.D.,For Those About To Rock We Salute You
7,Breaking The Rules,For Those About To Rock We Salute You
8,Night Of The Long Knives,For Those About To Rock We Salute You
9,Spellbound,For Those About To Rock We Salute You




SELECT
    t.Name AS TrackName,
    a.Title AS AlbumTitle
FROM
    tracks AS t
JOIN
    albums AS a ON t.AlbumId = a.AlbumId
LIMIT 20;




In [6]:
query = """
SELECT
    t.Name AS TrackName,
    SUM(ii.Quantity) AS TotalQuantitySold
FROM
    invoice_items AS ii
JOIN
    tracks AS t ON ii.TrackId = t.TrackId
GROUP BY
    t.Name
ORDER BY
    TotalQuantitySold DESC
LIMIT 10;
"""
display_results(query)

,TrackName,TotalQuantitySold
0,The Trooper,5
1,Untitled,4
2,The Number Of The Beast,4
3,Sure Know Something,4
4,Hallowed Be Thy Name,4
5,Eruption,4
6,Where Eagles Dare,3
7,Welcome Home (Sanitarium),3
8,Sweetest Thing,3
9,Surrender,3




SELECT
    t.Name AS TrackName,
    SUM(ii.Quantity) AS TotalQuantitySold
FROM
    invoice_items AS ii
JOIN
    tracks AS t ON ii.TrackId = t.TrackId
GROUP BY
    t.Name
ORDER BY
    TotalQuantitySold DESC
LIMIT 10;




In [7]:
query = """
SELECT
    t.Name AS TrackName,
    ii.Quantity
FROM
    invoice_items AS ii
JOIN
    tracks AS t ON ii.TrackId = t.TrackId
LIMIT 10;
"""
display_results(query)

,TrackName,Quantity
0,Balls to the Wall,1
1,Restless and Wild,1
2,Put The Finger On You,1
3,Inject The Venom,1
4,Evil Walks,1
5,Breaking The Rules,1
6,Dog Eat Dog,1
7,Overdose,1
8,Love In An Elevator,1
9,Janie's Got A Gun,1




SELECT
    t.Name AS TrackName,
    ii.Quantity
FROM
    invoice_items AS ii
JOIN
    tracks AS t ON ii.TrackId = t.TrackId
LIMIT 10;




In [8]:
query = """
SELECT
    ar.Name AS ArtistName,
    SUM(ii.Quantity) AS TotalQuantitySold
FROM
    artists AS ar
JOIN
    albums AS al ON ar.ArtistId = al.ArtistId
JOIN
    tracks AS t ON al.AlbumId = t.AlbumId
JOIN
    invoice_items AS ii ON t.TrackId = ii.TrackId
GROUP BY
    ar.Name
ORDER BY
    TotalQuantitySold DESC
LIMIT 10;
"""
display_results(query)

,ArtistName,TotalQuantitySold
0,Iron Maiden,140
1,U2,107
2,Metallica,91
3,Led Zeppelin,87
4,Os Paralamas Do Sucesso,45
5,Deep Purple,44
6,Faith No More,42
7,Lost,41
8,Eric Clapton,40
9,R.E.M.,39




SELECT
    ar.Name AS ArtistName,
    SUM(ii.Quantity) AS TotalQuantitySold
FROM
    artists AS ar
JOIN
    albums AS al ON ar.ArtistId = al.ArtistId
JOIN
    tracks AS t ON al.AlbumId = t.AlbumId
JOIN
    invoice_items AS ii ON t.TrackId = ii.TrackId
GROUP BY
    ar.Name
ORDER BY
    TotalQuantitySold DESC
LIMIT 10;


